In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

%matplotlib inline

import os

In [ ]:
DATA_ROOT = 'Data'
DB_NAME = 'amiunique-1month-aug17.sqlite3'

PATH_TO_DB = os.path.join(DATA_ROOT, DB_NAME)

In [ ]:
con = sqlite3.connect(PATH_TO_DB)
df = pd.read_sql_query('select * from fpData', con)